미니배치, 이터레이션, 에포크 참조  
https://wikidocs.net/55580

In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더
from torch.utils.data import Dataset

In [39]:
class TensorDataset(Dataset):
    def __init__(self):
        self.x_data = [[73, 80, 75],
                       [93, 88, 93],
                       [89, 91, 90],
                       [96, 98, 100],
                       [73, 66, 70]]
        self.y_data = [[152], [185], [180], [196], [142]]

      # 총 데이터의 개수를 리턴
    def __len__(self): 
        return len(self.x_data)

      # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
    def __getitem__(self, idx): 
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
        return x, y

In [40]:
class DiabetesDataset(Dataset):
    
    def __init__(self):
        # 커스텀 데이터셋 클래스의 생성자
        # 데이터를 불러오고 전처리 한다.
        xy = np.loadtxt('./data/diabetes.csv.gz', delimiter=',', dtype=np.float32)
        self.x_data = torch.from_numpy(xy[:, 0:-1])
        self.y_data = torch.from_numpy(xy[:, [-1]])
        print(f'X\'s shape: {self.x_data.shape} | Y\'s shape: {self.y_data.shape}')

    def __len__(self):
        # 데이터셋의 길이를 반환하는 메소드
        return len(self.x_data)
        
    def __getitem__(self, idx):
        # 데이터셋에서 하나의 특정 샘플 (x, y)을 가져오는 메소드
        return self.x_data[idx], self.y_data[idx]

In [41]:
# 구현한 커스텀 데이터셋 클래스의 인스턴스 생성
dataset = DiabetesDataset()

X's shape: torch.Size([759, 8]) | Y's shape: torch.Size([759, 1])


In [47]:
# DataLoader 클래스를 사용하여 학습에 사용되는 객체 생성 
dataloader = DataLoader(dataset, batch_size=100, shuffle=True,
                       num_workers=0)

## 퀴즈 (Easy)  
당뇨병 데이터셋에 대해서 로지스틱 회귀 모델을 구현한다면 첫번째 레이어의 노드 수는 몇개가 되어야 할까요??  답변해보세요!!  

## 퀴즈 (Normal)  
이전의 노트북 파일을 참고해서 LogisticRegressionModel 클래스를 구현하세요.  
1) 생성자 :  
모델의 은닉층 수는 3층이고 은닉층마다 (8, 6, 4) 개의 뉴런을 가집니다.  
마지막 레이어에는 시그모이드 모듈을 이용해 비선형변환을 수행하세요.  
2) forward :   
최종적으로 확률 값을 예측하도록 레이어를 쌓아서 y를 반환하세요.  

In [48]:
class LogisticRegressionModel(nn.Module):
    def __init__(self):
        super(LogisticRegressionModel, self).__init__()
        self.L1 = nn.Linear(8, 6)
        self.L2 = nn.Linear(6, 4)
        self.L3 = nn.Linear(4, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        z1 = self.sigmoid(self.L1(x))
        z2 = self.sigmoid(self.L2(z1))
        z3 = self.sigmoid(self.L3(z2))
        y_pred = self.sigmoid(z3)
        return y_pred

model = LogisticRegressionModel()

## 퀴즈 (Easy)  
위 모델을 학습시키기 위해서는 어떤 손실함수를 선택해야할까요?? 

In [49]:
bce_loss = nn.BCELoss(reduction='mean')

learning_rate = 0.01
optimizer = optim.SGD(model.parameters(), lr=learning_rate)


## 퀴즈 (Easy)  
반복문에서 enumerate를 반드시 사용해야합니다.  
그 이유가 무엇일까요?  

In [58]:


epochs = 10

for epoch in range(epochs):
    for batch_idx, samples in enumerate(dataloader):
        x_train, y_train = samples
        y_pred = model(x_train)
        loss = bce_loss(y_pred, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f'Epoch: {epoch+1:4d}/{epochs} Batch {batch_idx+1:4d}/{len(dataloader)} \
                Loss: {loss.item():4f}')


c:\users\ye200\anaconda3\envs\env\lib\site-packages\ipykernel_launcher.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Epoch:    1/10 Batch    1/8                 Loss: 0.642540
Epoch:    1/10 Batch    2/8                 Loss: 0.614771
Epoch:    1/10 Batch    3/8                 Loss: 0.625879
Epoch:    1/10 Batch    4/8                 Loss: 0.670300
Epoch:    1/10 Batch    5/8                 Loss: 0.637009
Epoch:    1/10 Batch    6/8                 Loss: 0.698061
Epoch:    1/10 Batch    7/8                 Loss: 0.648008
Epoch:    1/10 Batch    8/8                 Loss: 0.623117
Epoch:    2/10 Batch    1/8                 Loss: 0.664765
Epoch:    2/10 Batch    2/8                 Loss: 0.614765
Epoch:    2/10 Batch    3/8                 Loss: 0.648163
Epoch:    2/10 Batch    4/8                 Loss: 0.636842
Epoch:    2/10 Batch    5/8                 Loss: 0.637043
Epoch:    2/10 Batch    6/8                 Loss: 0.686935
Epoch:    2/10 Batch    7/8                 Loss: 0.647997
Epoch:    2/10 Batch    8/8                 Loss: 0.623160
Epoch:    3/10 Batch    1/8                 Loss: 0.6757